<a href="https://colab.research.google.com/github/WoradeeKongthong/medical_cost_regression/blob/master/02_Medical_Cost_Multiple_Linear_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd

In [0]:
## Data Preprocessing

# importing the dataset
df = pd.read_csv('/content/drive/My Drive/life-long learning/MyProjects/Medical Cost/cleaned_insurance.csv')

In [3]:
df.head()

,age,sex,bmi,children,smoker,region,charges,bmi_status,AgeInterval
0,19,female,27.900,0,True,southwest,16884.92400,overweight,"(15, 25]"
1,18,male,33.770,1,False,southeast,1725.55230,obesity,"(15, 25]"
2,28,male,33.000,3,False,southeast,4449.46200,obesity,"(25, 45]"
3,33,male,22.705,0,False,northwest,21984.47061,ideal,"(25, 45]"
4,32,male,28.880,0,False,northwest,3866.85520,overweight,"(25, 45]"


In [0]:
X = df.iloc[:, :6].values
y = df.iloc[:, 6].values

In [28]:
print(X)

[[19 'female' 27.9 0 True 'southwest']
 [18 'male' 33.77 1 False 'southeast']
 [28 'male' 33.0 3 False 'southeast']
 ...
 [18 'female' 36.85 0 False 'southeast']
 [21 'female' 25.8 0 False 'southwest']
 [61 'female' 29.07 0 True 'northwest']]


In [29]:
print(y)

[16884.92  1725.55  4449.46 ...  1629.83  2007.94 29141.36]


In [0]:
# encoding categorical data
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(drop='first'), [1,4,5])], remainder='passthrough')
X = ct.fit_transform(X)

In [31]:
print(X[:5])

[[0.0 1.0 0.0 0.0 1.0 19 27.9 0]
 [1.0 0.0 0.0 1.0 0.0 18 33.77 1]
 [1.0 0.0 0.0 1.0 0.0 28 33.0 3]
 [1.0 0.0 1.0 0.0 0.0 33 22.705 0]
 [1.0 0.0 1.0 0.0 0.0 32 28.88 0]]


In [0]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

In [33]:
# Training the Multiple Linear Regression model on the Training set
from sklearn.linear_model import LinearRegression
regressor = LinearRegression()
regressor.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [34]:
# Predicting the Test set results
y_pred = regressor.predict(X_test)
np.set_printoptions(precision=2)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1)[:10])

[[33485.72 40003.33]
 [37383.74 45863.21]
 [ 9723.58  2927.06]
 [10668.63  7348.14]
 [-1615.23  1694.8 ]
 [31343.68 37829.72]
 [10420.5   7445.92]
 [ 8129.98  7144.86]
 [ 4993.3   4992.38]
 [ 7326.47 14358.36]]


In [45]:
# Model Evaluation
from sklearn.metrics import r2_score,explained_variance_score
print(r2_score(y_test, y_pred))
print(explained_variance_score(y_test, y_pred))

0.7607702377546052
0.7607761428847546


In [49]:
# Applying K-Fold Cross Validation
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = regressor, X = X_train, y = y_train, cv = 10)
print("Accuracy: {:.2f} %".format(accuracies.mean()*100))
print("Standard Deviation: {:.2f} %".format(accuracies.std()*100))

Accuracy: 73.22 %
Standard Deviation: 7.52 %


In [36]:
## (PLUS) Building the optimal model using Backward Elimination method
from statsmodels.regression.linear_model import OLS
from statsmodels.tools.tools import add_constant
X_train = add_constant(X_train)
X_train = X_train.astype(float)
y_train = y_train.tolist()

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
X_opt = X_train[:,[0,1,2,3,4,5,6,7,8]]

In [38]:
model = OLS(y_train, X_opt)
results = model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.748
Model:                            OLS   Adj. R-squared:                  0.746
Method:                 Least Squares   F-statistic:                     392.7
Date:                Sat, 09 May 2020   Prob (F-statistic):          7.36e-311
Time:                        16:26:39   Log-Likelihood:                -10828.
No. Observations:                1070   AIC:                         2.167e+04
Df Residuals:                    1061   BIC:                         2.172e+04
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       -1.11e+04   1113.086     -9.974      0.000   -1.33e+04   -8918.330
x1          -130.3474    370.880     -0.351      0.725    -858.089     597.394
x2          2.367e+04    461.172     51.335      0.000    2.28e+04    2.46e+04
x3          -382.8277    528.045     -0.725      0.469   -1418.959     653.304
x4          -701.6288    532.948     -1.317      0.188   -1747.381     344.124
x5          -826.6410    537.217     -1.539      0.124   -1880.769     227.487
x6           255.5530     13.274     19.252      0.000     229.507     281.599
x7           314.3859     32.553      9.658      0.000     250.510     378.261
x8           385.2357    152.922      2.519      0.012      85.173     685.299
==============================================================================
Omnibus:                      245.325   Durbin-Watson:                   2.093
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              582.531
Skew:                           1.229   Prob(JB):                    3.20e-127
Kurtosis:                       5.650   Cond. No.                         314.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [0]:
# eliminate X_train[1] : gender
X_opt = X_train[:,[0,2,3,4,5,6,7,8]]

In [40]:
model = OLS(y_train, X_opt)
results = model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.748
Model:                            OLS   Adj. R-squared:                  0.746
Method:                 Least Squares   F-statistic:                     449.2
Date:                Sat, 09 May 2020   Prob (F-statistic):          3.56e-312
Time:                        16:27:02   Log-Likelihood:                -10828.
No. Observations:                1070   AIC:                         2.167e+04
Df Residuals:                    1062   BIC:                         2.171e+04
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -1.116e+04   1102.193    -10.122      0.000   -1.33e+04   -8993.158
x1          2.366e+04    460.011     51.442      0.000    2.28e+04    2.46e+04
x2          -385.8023    527.759     -0.731      0.465   -1421.372     649.768
x3          -700.0353    532.709     -1.314      0.189   -1745.317     345.247
x4          -822.9362    536.892     -1.533      0.126   -1876.425     230.552
x5           255.7762     13.253     19.299      0.000     229.770     281.782
x6           313.7323     32.486      9.657      0.000     249.987     377.477
x7           384.7246    152.851      2.517      0.012      84.799     684.650
==============================================================================
Omnibus:                      245.233   Durbin-Watson:                   2.093
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              581.609
Skew:                           1.229   Prob(JB):                    5.07e-127
Kurtosis:                       5.646   Cond. No.                         311.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [0]:
# eliminate X_train[3] : region northwest
X_opt = X_train[:,[0,2,4,5,6,7,8]]

In [42]:
model = OLS(y_train, X_opt)
results = model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.747
Model:                            OLS   Adj. R-squared:                  0.746
Method:                 Least Squares   F-statistic:                     524.2
Date:                Sat, 09 May 2020   Prob (F-statistic):          1.95e-313
Time:                        16:27:17   Log-Likelihood:                -10828.
No. Observations:                1070   AIC:                         2.167e+04
Df Residuals:                    1063   BIC:                         2.171e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -1.136e+04   1066.496    -10.650      0.000   -1.35e+04   -9265.945
x1          2.368e+04    459.370     51.549      0.000    2.28e+04    2.46e+04
x2          -504.5036    460.588     -1.095      0.274   -1408.269     399.262
x3          -625.3668    463.805     -1.348      0.178   -1535.443     284.710
x4           255.7794     13.250     19.303      0.000     229.779     281.779
x5           313.9084     32.478      9.665      0.000     250.179     377.637
x6           382.0107    152.773      2.501      0.013      82.240     681.781
==============================================================================
Omnibus:                      245.017   Durbin-Watson:                   2.094
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              579.275
Skew:                           1.230   Prob(JB):                    1.63e-126
Kurtosis:                       5.636   Cond. No.                         296.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [0]:
# Note : eliminating independent variables does not improve Adjusted R-square